# Post Here: Subreddit Predictor

## Recommendation API - 1.4

> aka: the MVP Classifier® Forest

---
---

## Intro - MVP Classifier (Model #4)

The fourth (at least) iteration of the model for recommending (predicting) appropriate subreddit(s).

The model will be trained using the [reddit self-post classification task dataset](https://www.kaggle.com/mswarbrickjones/reddit-selfposts), available on Kaggle thanks to [Evolution AI](https://evolution.ai//blog/page/5/an-imagenet-like-text-classification-task-based-on-reddit-posts/).

The full dataset includes 1,013,000 rows (1000 records each from 1013 subreddits).

---

### Imports

In [1]:
# === General imports === #
import pandas as pd
import numpy as np
import os

In [2]:
# === sklearn imports === #
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

---

### Load and preprocess the data

In [3]:
# === Load the saved version === #
df1 = pd.read_csv("rspct_100k.csv", sep="\t")

# === First looks === #
print(df1.shape)
df1.head()

(100000, 4)


,id,subreddit,title,selftext
0,0,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi..."
1,1,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...
2,2,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
3,3,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...
4,4,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,..."


In [4]:
# === Split up dataset into train and test === #
# from sklearn.model_selection import train_test_split

# 80% train, 20% test, stratified on the target
train, test = train_test_split(df1, test_size=0.2)

train.shape, test.shape

((80000, 4), (20000, 4))

In [5]:
# === Arrange data into feature and target === #

# MVP model only uses 'selftext' feature
X_train = train["selftext"]
X_test = test["selftext"]

# Predict the subreddit of each post
y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(80000,) (20000,)
(80000,) (20000,)


In [6]:
# === Encode the target using LabelEncoder === #

# This process naively transforms each class of the target into a number
# from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # Instantiate a new encoder instance
le.fit(y_train)  # Fit it on training label data

# Transform both using the train-fit instance
y_train = le.transform(y_train)
y_test  = le.transform(y_test)

y_train[:8]

array([565, 179, 699,  76, 363, 824, 284, 828])

---
---

### Vectorization

In [8]:
# === Vectorize! === #

# Extract features from the text data using bag-of-words (single words + bigrams).
# Uses tfidf weighting (helps a little for Naive Bayes in general).
# from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=100000,
    min_df=10,
    ngram_range=(1,2),
    stop_words=None,  # TODO: try out spacy's or sklearn's stopwords
)

# Fit the vectorizer on the feature column to create vocab (doc-term matrix)
vocab = tfidf.fit(X_train)

# Get sparse document-term matrices
X_train_sparse = vocab.transform(X_train)
X_test_sparse = vocab.transform(X_test)

In [9]:
# === Feature Selection === #

from sklearn.feature_selection import chi2, SelectKBest

selector = SelectKBest(chi2, 3000)

selector.fit(X_train_sparse, y_train)

X_train_select = selector.transform(X_train_sparse)
X_test_select  = selector.transform(X_test_sparse)

X_train_select.shape, X_test_select.shape

((80000, 3000), (20000, 3000))

### Baseline Random Forest

In [24]:
# === Baseline RandomForest model === #
from sklearn.ensemble import RandomForestClassifier

# Instantiate and train the model
rfc = RandomForestClassifier(max_depth=32, n_jobs=-1, n_estimators=200)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=32, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [25]:
# === Create predictions on test feature === #
y_pred_proba_rfc = rfc.predict_proba(X_test)

# === For each prediction, find the index with the highest probability === #
y_pred_rfc = np.argmax(y_pred_proba_rfc, axis=1)
y_pred_rfc[:10]

array([122, 925, 842, 330, 402, 870,  48, 842, 402, 842])

In [26]:
# === Evaluate performance using precision-at-k === #

def precision_at_k(y_true, y_pred, k=5):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.argsort(y_pred, axis=1)
    y_pred = y_pred[:, ::-1][:, :k]
    arr = [y in s for y, s in zip(y_true, y_pred)]
    return np.mean(arr)

print('precision@1 =', np.mean(y_test == y_pred_rfc))
print('precision@3 =', precision_at_k(y_test, y_pred_proba_rfc, 3))
print('precision@5 =', precision_at_k(y_test, y_pred_proba_rfc, 5))

precision@1 = 0.27855
precision@3 = 0.319
precision@5 = 0.32755


Even with a pretty solid Random Forest, the results are not great.

Let's see what the predictions look like.

In [30]:
# === Function to serve predictions - Random Forest === #


def predict(post: str, n: int = 5) -> dict:
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    post : string
        Selftext that needs a home.
    n    : integer
        The desired name of the output file,
        not including the '.pkl' extension.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    
    # Vectorize the post -> sparse doc-term matrix
    post_vec = vocab.transform([post])
    
    # Generate predicted probabilities from trained model
    proba = rfc.predict_proba(post_vec)
    
    # Wrangle into correct format
    return (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "subreddit", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:n]  # n-top predictions to serve
                .to_dict(orient="records")
               )

---

### Naive Bayes Classifier

In [10]:
# === Naive Bayes model === #
# from sklearn.naive_bayes import MultinomialNB

# Instantiate and train the model
nb = MultinomialNB(alpha=0.4)
nb.fit(X_train_select, y_train)

MultinomialNB(alpha=0.4, class_prior=None, fit_prior=True)

In [11]:
# === Create predictions on test feature === #
y_pred_proba = nb.predict_proba(X_test_select)

# === For each prediction, find the index with the highest probability === #
# import numpy as np
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred[:10]

array([104,  95, 240, 198, 714, 872, 177, 104, 507, 343])

In [12]:
def precision_at_k(y_true, y_pred, k=5):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.argsort(y_pred, axis=1)
    y_pred = y_pred[:, ::-1][:, :k]
    arr = [y in s for y, s in zip(y_true, y_pred)]
    return np.mean(arr)

print('precision@1 =', np.mean(y_test == y_pred))
print('precision@3 =', precision_at_k(y_test, y_pred_proba, 3))
print('precision@5 =', precision_at_k(y_test, y_pred_proba, 5))

precision@1 = 0.35865
precision@3 = 0.44935
precision@5 = 0.47965


---

### Predict subreddit from new input

Now that our model is trained and we have our baseline, we can use it to predict what subreddit would belong to a new piece of data (a post).

In [43]:
# === Function to serve predictions - Naive Bayes === #


def predict(post: str, n: int = 5) -> dict:
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    post : string
        Selftext that needs a home.
    n    : integer
        The desired name of the output file,
        not including the '.pkl' extension.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    
    # Vectorize the post -> sparse doc-term matrix
    post_vec = vocab.transform([post])
    
    # Generate predicted probabilities from trained model
    proba = nb.predict_proba(post_vec)
    
    # Wrangle into correct format
    return (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "subreddit", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:n]  # n-top predictions to serve
                .to_dict(orient="records")
               )

In [13]:
# === Function to serve predictions === #
# The main functionality of the predict API endpoint

def predict(post: str, n: int = 5):
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    post : string
        Selftext that needs a home.
    n    : integer
        The desired name of the output file,
        not including the '.pkl' extension.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    
    # Vectorize the post -> sparse doc-term matrix
    post_sparse = vocab.transform([post])
    
    # Feature selection
    post_select = selector.transform(post_sparse)
    
    # Generate predicted probabilities from trained model
    proba = nb.predict_proba(post_select)
    
    # Wrangle into correct format
    return (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "subreddit", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:n]  # n-top predictions to serve
                .to_dict(orient="records")
               )

In [14]:
post_science = """Is there an evolutionary benefit to eating spicy food that lead to consumption across numerous cultures throughout history? Or do humans just like the sensation?

I love spicy food and have done ever since I tried it. By spicy I mean HOT, like chilli peppers (we say spicy in England, I don't mean to state the obvious I'm just not sure if that's a global term and I've assumed too much before). I love a vast array of spicy foods from all around the world. I was just wondering if there was some evolutionary basis as to why spicy food managed to become some widely consumed historically. Though there seem to

It way well be that we just like a tingly mouth, the simple things in life."""

science_recs = predict(post_science)
science_recs

[{'subreddit': 'HotPeppers', 'proba': 0.0015314459001810564},
 {'subreddit': 'DragonsDogma', 'proba': 0.0013730010156944878},
 {'subreddit': 'cosplay', 'proba': 0.0013357695514130226},
 {'subreddit': 'radiohead', 'proba': 0.0013233044911954515},
 {'subreddit': 'Charity', 'proba': 0.0012862958864824758}]

In [15]:
# === Test post from r/buildmeapc === #

post_pc = """I posted my wants for my build about 2 months ago. Ordered them and when I went to build it I was soooooo lost. It took 3 days to put things together because I was afraid I would break something when I finally got the parts together it wouldn’t start, I was so defeated. With virtually replacing everything yesterday it finally booted and I couldn’t be more excited!"""

post_pc_recs = predict(post_pc, 10)
post_pc_recs

[{'subreddit': 'DragonsDogma', 'proba': 0.0013750000000000001},
 {'subreddit': 'cosplay', 'proba': 0.001337499999999999},
 {'subreddit': 'radiohead', 'proba': 0.0013249999999999994},
 {'subreddit': 'Charity', 'proba': 0.0012874999999999998},
 {'subreddit': 'titanfall', 'proba': 0.0012749999999999992},
 {'subreddit': 'Chromecast', 'proba': 0.0012749999999999992},
 {'subreddit': 'snapchat', 'proba': 0.0012500000000000002},
 {'subreddit': '3Dprinting', 'proba': 0.0012500000000000002},
 {'subreddit': 'resumes', 'proba': 0.0012500000000000002},
 {'subreddit': 'sales', 'proba': 0.0012374999999999995}]

In [16]:
# === Example post from 'r/learnprogramming' === #

post = """I am a new grad looking for a job and currently in the process with a company for a junior backend engineer role. I was under the impression that the position was Javascript but instead it is actually Java. My general programming and "leet code" skills are pretty good, but my understanding of Java is pretty shallow. How can I use the next three days to best improve my general Java knowledge? Most resources on the web seem to be targeting complete beginners. Maybe a book I can skim through in the next few days?

Edit:

A lot of people are saying "the company is a sinking ship don't even go to the interview". I just want to add that the position was always for a "junior backend engineer". This company uses multiple languages and the recruiter just told me the incorrect language for the specific team I'm interviewing for. I'm sure they're mainly interested in seeing my understanding of good backend principles and software design, it's not a senior lead Java position."""

# === Test out the function === #
post_pred = predict(post)  # Default is 5 results
post_pred

[{'subreddit': 'javahelp', 'proba': 0.0023090280144774347},
 {'subreddit': 'androiddev', 'proba': 0.0017914570853405122},
 {'subreddit': 'cscareerquestions', 'proba': 0.001611167703091607},
 {'subreddit': 'interviews', 'proba': 0.0014772532994282328},
 {'subreddit': 'DragonsDogma', 'proba': 0.0013503849085304746}]

In [17]:
# === Test it out with another dummy post === #

# This one comes from r/suggestmeabook
post2 = """I've been dreaming about writing my own stort story for a while but I want to give it an unexpected ending. I've read lots of books, but none of them had the plot twist I want. I want to read books with the best plot twists, so that I can analyze what makes a good plot twist and write my own story based on that points. I don't like romance novels and I mostly enjoy sci-fi or historical books but anything beside romance novels would work for me, it doesn't have to be my type of novel. I'm open to experience after all. I need your help guys. Thanks in advance."""

# === This time with 10 results === #
post2_pred = predict(post2, n=10)
post2_pred

[{'subreddit': 'DragonsDogma', 'proba': 0.0013712105334414554},
 {'subreddit': 'radiohead', 'proba': 0.0013476440520005367},
 {'subreddit': 'cosplay', 'proba': 0.0013340494974593818},
 {'subreddit': 'MaladaptiveDreaming', 'proba': 0.0012971874649116608},
 {'subreddit': 'Charity', 'proba': 0.0012846867326412827},
 {'subreddit': 'titanfall', 'proba': 0.001272269133622847},
 {'subreddit': 'Chromecast', 'proba': 0.0012709935975766793},
 {'subreddit': 'snapchat', 'proba': 0.0012473813271700195},
 {'subreddit': 'resumes', 'proba': 0.00124646426422597},
 {'subreddit': '3Dprinting', 'proba': 0.0012457958646503995}]

---

### Pickle Time

In [29]:
# === Create pickle func to make pickling (a little) easier === #

def picklizer(to_pickle, filename, path):
    """
    Creates a pickle file.
    
    Parameters
    ----------
    to_pickle : Python object
        The trained / fitted instance of the 
        transformer or model to be pickled.
    filename : string
        The desired name of the output file,
        not including the '.pkl' extension.
    path : string or path-like object
        The path to the desired output directory.
    """
    import os
    import pickle

    # Create the path to save location
    picklepath = os.path.join(path, filename)

    # Use context manager to open file
    with open(picklepath, "wb") as p:
        pickle.dump(to_pickle, p)

In [30]:
# === Picklize! === #
filepath = "./pickles"  # Change this accordingly

# Export LabelEncoder as pickle
picklizer(le, "05_le.pkl", filepath)

# Export selector as pickle
picklizer(selector, "05_selector.pkl", filepath)

# Export vectorizer as pickle
picklizer(vocab, "05_vocab.pkl", filepath)

# Export naive bayes model as pickle
picklizer(nb, "05_nb.pkl", filepath)

Load and consume pickles...

> _Enjoy responsibly._

In [68]:
# === Load the trained vectorizer === #
import pickle

le_path = os.path.join(filepath, "04_le.pkl")

# Use context manager to open and load pickle
with open(le_path, "rb") as p:
    le = pickle.load(p)

In [69]:
# === Load the trained vectorizer === #
import pickle

vocab_path = os.path.join(filepath, "04_vocab.pkl")

# Use context manager to open and load pickle
with open(vocab_path, "rb") as p:
    vocab = pickle.load(p)

In [70]:
# === Load the somewhat-trained Random Forest classifier === #
# import pickle

rfc_path = os.path.join(filepath, "04_rfc.pkl")

# Use context manager to open and load pickle
with open(rfc_path, "rb") as p:
    rfc = pickle.load(p)

In [71]:
# === Test out the pickled versions === #

# This one comes from r/buildmeapc
post3 = """I posted my wants for my build about 2 months ago. Ordered them and when I went to build it I was soooooo lost. It took 3 days to put things together because I was afraid I would break something when I finally got the parts together it wouldn’t start, I was so defeated. With virtually replacing everything yesterday it finally booted and I couldn’t be more excited!"""

# This time I'll pass in 20, because why not?
post3_recs = predict(post3, 20)
post3_recs

[{'subreddit': 'parrots', 'proba': 0.00211370360467277},
 {'subreddit': 'flexibility', 'proba': 0.002096921513234382},
 {'subreddit': 'bladeandsoul', 'proba': 0.002071709687128192},
 {'subreddit': 'Rowing', 'proba': 0.0020525020657867544},
 {'subreddit': 'StudentLoans', 'proba': 0.0020488032050522157},
 {'subreddit': 'gigantic', 'proba': 0.0019414817435439607},
 {'subreddit': 'PLC', 'proba': 0.0018997017900174518},
 {'subreddit': 'TransDIY', 'proba': 0.001892048211568393},
 {'subreddit': 'breastfeeding', 'proba': 0.0018844645436299533},
 {'subreddit': 'RocketLeague', 'proba': 0.0018758224984953434},
 {'subreddit': 'benzodiazepines', 'proba': 0.0018668138495396874},
 {'subreddit': 'The100', 'proba': 0.0018580496088653011},
 {'subreddit': 'minecraftsuggestions', 'proba': 0.0018453927006669075},
 {'subreddit': 'techsupport', 'proba': 0.0018442340100311113},
 {'subreddit': 'Blink182', 'proba': 0.0018239715732455283},
 {'subreddit': 'dpdr', 'proba': 0.0018163028805450827},
 {'subreddit': 'l